In [1]:
from base64 import encode
import spacy
from spacy.matcher import Matcher
import hashlib
import json
import numpy as np
from sklearn.metrics import precision_recall_fscore_support


nlp = spacy.load("en_core_web_sm")


/home/simret/miniconda3/envs/py39/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
def expand_working_list(pat):
    result = []
    if (pat == None):
        return []
    else:
        working_list = []
        optionals = []
        combinations = pat.split("+")
        for pattern in combinations:
            patterns_within = pattern.split("|")
            if (len(patterns_within) > 1):
                # result.append(working_list)
                # working_list = []
                optional_patterns = []
                for p in patterns_within:
                    if (p[0] == "["):
                        temp = {"LEMMA": {"IN": [p[1:-1]]}, "OP": "+"}
                        optional_patterns.append(temp)
                    elif (p[0] == "$"):
                        temp = {"ENT_TYPE": p[1:], "OP": "+"}
                        optional_patterns.append(temp)
                    else:
                        temp = {"POS": p, "OP": "+"}
                        optional_patterns.append(temp)
                count = len(working_list)
                if (count):
                    while (count):
                        count -= 1
                        temp = working_list.pop(0)
                        for opt in optional_patterns:
                            updated_pattern = temp + [opt]
                            working_list.append(updated_pattern)
                else:
                    working_list = [[x] for x in optional_patterns]

                # result.append(optional_patterns)
            else:
                if (pattern == "*"):
                    temp = {"OP": "?"}
                    # temp = {"OP": "?"}
                elif (pattern[0] == "["):
                    temp = {"LEMMA": {"IN": [pattern[1:-1]]}, "OP": "+"}
                elif (pattern[0] == "$"):
                    temp = {"ENT_TYPE": pattern[1:], "OP": "+"}
                else:
                    temp = {"POS": pattern, "OP": "+"}
                if (len(working_list) == 0):
                    working_list.append([temp])
                else:
                    for i in range(len(working_list)):
                        working_list[i].append(temp)
        return working_list

In [3]:

def check_matching(sent, working_list, explain=False):
    matcher = Matcher(nlp.vocab)
    for index, patterns in enumerate(working_list):
        matcher.add(f"rule{index}", [patterns])
    doc = nlp(sent)
    for i in doc:
        print(i)
    print(len(doc[:]))
    matches = matcher(doc)
    if(matches is not None and len(matches)>0):
        for id, start, end in matches:
            if(str(doc[start:end]).strip() !=""):
                if(explain):
                    return (True, [str(doc[start:end]).strip(), start, end])
                return True
    if(explain):
        return (False, "")
    return False

In [4]:
sent = "Unbelievable, it was me & my mother so it was a total of 100$ i could have gone to red lobster and ate better quality of food."
pattern = "NUM+*"

In [6]:
def tokenizer(sent):
    s = nlp(sent)
    new_sent = ""
    for i in s:
        new_sent +=f"{i} "
    return new_sent

In [8]:
new_sent = tokenizer(sent=sent)

In [10]:
new_sent.split(" ")[14]

'100'

In [5]:
working_list = expand_working_list(pattern)

In [45]:

check_matching(sent, working_list, explain=True)


Unbelievable
,
it
was
me
&
my
mother
so
it
was
a
total
of
100
$
i
could
have
gone
to
red
lobster
and
ate
better
quality
of
food
.
30


(True, ['100', 14, 15])

In [37]:
len(sent.split(" "))

27

In [46]:
sent.split(" ")[13]

'100$'